## Experiment tracking for XGBoost projects

In [ ]:
pip install -U xgboost

In [ ]:
pip install -U layer

In [3]:
import layer
from layer.decorators import model, dataset, resources,pip_requirements, fabric
from layer import Dataset

In [ ]:
layer.login()

In [ ]:
layer.init("xgboost")

In [ ]:
!git clone https://github.com/mwitiderrick/experiment-tracking-for-XGBoost-projects.git
!mv /content/experiment-tracking-for-XGBoost-projects/heart.csv heart.csv 

In [8]:
data_file = 'heart.csv'
@resources(data_file)
@dataset("heart")
def save_data():
  import pandas as pd 
  df = pd.read_csv(data_file)
  return df

In [9]:
import xgboost as xgb
from typing import cast
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
class XGBoostCallback(xgb.callback.TrainingCallback):
    def __init__(self,importance_type: str = "gain"):
      self.importance_type = importance_type

    def before_training(self, model):
      return model

    def after_training(self, model):
      if model.attr("best_score") is not None:
            print(
                {
                    "best_score": float(cast(str, model.attr("best_score"))),
                    "best_iteration": int(cast(str, model.attr("best_iteration"))),
                }
            )


      features = model.get_score(importance_type=self.importance_type)
      importance = [[feature, features[feature]] for feature in features]

      df = pd.DataFrame(data=importance,columns=["feature", "importance"])
      df = df.sort_values(by='importance', ascending=False)
      layer.log({"Feature Importance Table": df}) 
      plt.figure(figsize=(12,8))
      plt.xticks(rotation=60, fontsize = 20)
      sns.barplot(y=df['feature'], x=df['importance']) 

      layer.log({"Feature Importance": plt.gcf() })

      return model



In [10]:
@fabric("f-medium")
@model("xgboost",dependencies=[Dataset('heart')])
@pip_requirements(packages=["matplotlib","seaborn"])
def train():
  import xgboost as xgb
  import pandas as pd
  from sklearn.model_selection import train_test_split
  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn.metrics import accuracy_score
  from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
  from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve,precision_recall_curve


  df = layer.get_dataset("heart").to_pandas()
  X = df.drop(['target'],axis=1)
  y = df["target"]
  test_size = 0.3
  random_state = 13
  
  layer.log({"test_size":test_size,"random_state":random_state})
  params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
  layer.log(params)
  X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=test_size,random_state=random_state)
  xgb_model = xgb.XGBClassifier(**params,callbacks=[XGBoostCallback()])
  xgb_model.fit(X_train,y_train) 
  predictions = xgb_model.predict(X_test)
  
  layer.log({"Accuracy" : accuracy_score(predictions, y_test) })
  cm = confusion_matrix(y_test, predictions, labels=xgb_model.classes_)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb_model.classes_)
  disp.plot()
  layer.log({"Confusion metrics" : plt.gcf()})


  probs = xgb_model.predict(X_test)
  # Calculate average precision and area under the receiver operating characteric curve (ROC AUC)
  avg_precision = average_precision_score(y_test, probs, pos_label=1)
  auc = roc_auc_score(y_test, probs)
  layer.log({"AUC":f'{auc:.4f}'})
  layer.log({"avg_precision":f'{avg_precision:.4f}'})

  sample_preds = X_test
  sample_preds["predicted"] = predictions
  layer.log({"Sample predictions":sample_preds.head(100)})
  
  return xgb_model

In [ ]:
layer.run([save_data,train])

In [ ]:
import layer
import numpy as np
my_model = layer.get_model("layer/xgboost/models/xgboost").get_train()
predictions = my_model.predict(np.array([[68,1,2,180,274,1,0,150,1,1.6,1,0,3]]))
predictions

## Next steps
To learn more about using layer, you can: 
- Join our [Slack Community ](https://bit.ly/layercommunityslack)
- Visit [Layer Examples Repo](https://github.com/layerai/examples) for more examples
- Browse [Trending Layer Projects](https://layer.ai) on our mainpage
- Check out [Layer Documentation](https://docs.app.layer.ai) to learn more